In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from keras.layers import Dense, Dropout
from keras.models import Model, Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
import time

In [2]:
df = pd.read_csv("luka_dataset.csv",index_col=0)
df.head()

,G,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,...,BLK,TOV,PF,PTS,GmSc,"""+/-""",Win,Winby,Minutes,age_y
0,1,5,16,0.313,0,5,0.000,0,2,0.000,...,0,4,3,10,0.8,-18,0,-21,31.783333,19.632877
1,2,8,16,0.500,4,9,0.444,6,9,0.667,...,1,6,3,26,16.6,6,1,4,36.233333,19.641096
2,3,7,14,0.500,3,8,0.375,2,2,1.000,...,0,3,3,19,13.9,5,1,6,35.800000,19.646575
3,4,7,18,0.389,2,9,0.222,5,8,0.625,...,1,3,2,21,12.0,-12,0,-7,34.183333,19.652055
4,5,7,14,0.500,4,6,0.667,4,4,1.000,...,0,3,1,22,16.3,0,0,-9,35.150000,19.657534


In [3]:
df = df.astype('float32')

In [4]:
y = df['Win']
X = df.drop(columns=['Win','Winby'])

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,stratify=y, random_state=42)

In [6]:
def my_DL(epochs=6,batchsize=512):
    model = Sequential()
    model.add(Dense(32,activation='relu'))
    model.add(Dense(16,activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer='rmsprop', metrics=['accuracy'])
    return model

In [7]:
def train_hyper_tune(X,y):
    # create the pre-processing component
    my_scaler = StandardScaler()
    my_imputer = SimpleImputer(strategy="median")
    
    # define classifiers
    ## Classifier 1: Logistic Regression
    clf_LR = LogisticRegression(random_state=0,penalty='elasticnet',solver='saga')
    ## Classifier 2: Random Forest Classifier
    clf_RF = RandomForestClassifier(random_state=0)
    ## Classifier 3: Deep Learning Binary Classifier
    clf_DL = KerasClassifier(build_fn=my_DL)
    
    # define pipeline for three classifiers
    ## clf_LR
    pipe1 = Pipeline([('imputer', my_imputer), ('scaler', my_scaler), ('lr_model',clf_LR)])
    ## clf_RF
    pipe2 = Pipeline([('imputer', my_imputer), ('scaler', my_scaler), ('rf_model',clf_RF)])
    ## clf_DL
    pipe3 = Pipeline([('imputer', my_imputer), ('scaler', my_scaler), ('dl_model',clf_DL)])
    
    # create hyperparameter space of the three models
    ## clf_LR
    param_grid1 = {
        'lr_model__C' : [1e-1,1,10],
        'lr_model__l1_ratio' : [0,0.5,1]
    }
    ## clf_RF
    param_grid2 = {
        'rf_model__n_estimators' : [50,100],
        'rf_model__max_features' : [0.8,"auto"],
        'rf_model__max_depth' : [4,5]
    }
    ## clf_DL
    param_grid3 = {
        'dl_model__epochs' : [6,12,18,24],
        'dl_model__batchsize' : [256,512]
    }
    
    # set GridSearch via 5-fold cross-validation
    ## clf_LR
    grid1 = GridSearchCV(pipe1, cv=5, param_grid=param_grid1)
    ## clf_RF
    grid2 = GridSearchCV(pipe2, cv=5, param_grid=param_grid2)
    ## clf_DL
    grid3 = GridSearchCV(pipe3, cv=5, param_grid=param_grid3)
    
    # run the hyperparameter tunning process
    start_LR = time.perf_counter()
    grid1.fit(X,y)
    finish_LR = time.perf_counter()
    start_RF = time.perf_counter()
    grid2.fit(X,y)
    finish_RF = time.perf_counter()
    start_DL = time.perf_counter()
    grid3.fit(X,y)
    finish_DL = time.perf_counter()
    print("LR: ",finish_LR-start_LR)
    print("RF: ",finish_RF-start_RF)
    print("DL: ",finish_DL-start_DL)
    # return results of the tunning process
    return grid1,grid2,grid3,pipe1,pipe2,pipe3

In [8]:
my_grid1,my_grid2,my_grid3,my_pipe1,my_pipe2,my_pipe3 = train_hyper_tune(X_train, y_train)

/Users/rushipatel/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/rushipatel/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/rushipatel/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/rushipatel/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/rushipatel/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sa

Epoch 1/6
3/3 [==============================] - 0s 3ms/step - loss: 0.7729 - accuracy: 0.5227
Epoch 2/6
3/3 [==============================] - 0s 2ms/step - loss: 0.7211 - accuracy: 0.5341
Epoch 3/6
3/3 [==============================] - 0s 1ms/step - loss: 0.6895 - accuracy: 0.5682
Epoch 4/6
3/3 [==============================] - 0s 3ms/step - loss: 0.6649 - accuracy: 0.6136
Epoch 5/6
3/3 [==============================] - 0s 2ms/step - loss: 0.6455 - accuracy: 0.6250
Epoch 6/6
1/1 [==============================] - 0s 823us/step - loss: 0.6525 - accuracy: 0.6087
Epoch 1/6
3/3 [==============================] - 0s 1ms/step - loss: 0.7954 - accuracy: 0.4607
Epoch 2/6
3/3 [==============================] - 0s 1ms/step - loss: 0.7260 - accuracy: 0.4831
Epoch 3/6
3/3 [==============================] - 0s 2ms/step - loss: 0.6908 - accuracy: 0.5393
Epoch 4/6
3/3 [==============================] - 0s 3ms/step - loss: 0.6658 - accuracy: 0.5281
Epoch 5/6
3/3 [==============================] -

3/3 [==============================] - 0s 1ms/step - loss: 0.7271 - accuracy: 0.4944
Epoch 3/12
3/3 [==============================] - 0s 1ms/step - loss: 0.7070 - accuracy: 0.5056
Epoch 4/12
3/3 [==============================] - 0s 2ms/step - loss: 0.6915 - accuracy: 0.5169
Epoch 5/12
3/3 [==============================] - 0s 3ms/step - loss: 0.6796 - accuracy: 0.5506
Epoch 6/12
3/3 [==============================] - 0s 4ms/step - loss: 0.6690 - accuracy: 0.5730
Epoch 7/12
3/3 [==============================] - 0s 4ms/step - loss: 0.6602 - accuracy: 0.5955
Epoch 8/12
3/3 [==============================] - 0s 4ms/step - loss: 0.6515 - accuracy: 0.6180
Epoch 9/12
3/3 [==============================] - 0s 2ms/step - loss: 0.6421 - accuracy: 0.6404
Epoch 10/12
3/3 [==============================] - 0s 2ms/step - loss: 0.6340 - accuracy: 0.6404
Epoch 11/12
3/3 [==============================] - 0s 1ms/step - loss: 0.6266 - accuracy: 0.6517
Epoch 12/12
1/1 [==============================] 

1/1 [==============================] - 0s 1ms/step - loss: 0.5891 - accuracy: 0.6522
Epoch 1/18
3/3 [==============================] - 0s 939us/step - loss: 0.6978 - accuracy: 0.5393
Epoch 2/18
3/3 [==============================] - 0s 1ms/step - loss: 0.6696 - accuracy: 0.5955
Epoch 3/18
3/3 [==============================] - 0s 2ms/step - loss: 0.6517 - accuracy: 0.6180
Epoch 4/18
3/3 [==============================] - 0s 3ms/step - loss: 0.6386 - accuracy: 0.6292
Epoch 5/18
3/3 [==============================] - 0s 2ms/step - loss: 0.6259 - accuracy: 0.6629
Epoch 6/18
3/3 [==============================] - 0s 2ms/step - loss: 0.6155 - accuracy: 0.6966
Epoch 7/18
3/3 [==============================] - 0s 3ms/step - loss: 0.6048 - accuracy: 0.6854
Epoch 8/18
3/3 [==============================] - 0s 3ms/step - loss: 0.5936 - accuracy: 0.7303
Epoch 9/18
3/3 [==============================] - 0s 2ms/step - loss: 0.5835 - accuracy: 0.7303
Epoch 10/18
3/3 [==============================] 

3/3 [==============================] - 0s 4ms/step - loss: 0.6831 - accuracy: 0.5393
Epoch 5/18
3/3 [==============================] - 0s 5ms/step - loss: 0.6689 - accuracy: 0.5393
Epoch 6/18
3/3 [==============================] - 0s 6ms/step - loss: 0.6546 - accuracy: 0.5618
Epoch 7/18
3/3 [==============================] - 0s 5ms/step - loss: 0.6415 - accuracy: 0.5618
Epoch 8/18
3/3 [==============================] - 0s 3ms/step - loss: 0.6310 - accuracy: 0.6292
Epoch 9/18
3/3 [==============================] - 0s 2ms/step - loss: 0.6194 - accuracy: 0.6292
Epoch 10/18
3/3 [==============================] - 0s 5ms/step - loss: 0.6094 - accuracy: 0.6404
Epoch 11/18
3/3 [==============================] - 0s 5ms/step - loss: 0.5994 - accuracy: 0.6517
Epoch 12/18
3/3 [==============================] - 0s 3ms/step - loss: 0.5906 - accuracy: 0.6966
Epoch 13/18
3/3 [==============================] - 0s 4ms/step - loss: 0.5799 - accuracy: 0.7191
Epoch 14/18
3/3 [==============================

...

3/3 [==============================] - 0s 2ms/step - loss: 0.6661 - accuracy: 0.5843
Epoch 3/12
3/3 [==============================] - 0s 2ms/step - loss: 0.6452 - accuracy: 0.6067
Epoch 4/12
3/3 [==============================] - 0s 2ms/step - loss: 0.6286 - accuracy: 0.6404
Epoch 5/12
3/3 [==============================] - 0s 2ms/step - loss: 0.6166 - accuracy: 0.6517
Epoch 6/12
3/3 [==============================] - 0s 3ms/step - loss: 0.6053 - accuracy: 0.6742
Epoch 7/12
3/3 [==============================] - 0s 3ms/step - loss: 0.5930 - accuracy: 0.6854
Epoch 8/12
3/3 [==============================] - 0s 2ms/step - loss: 0.5828 - accuracy: 0.6854
Epoch 9/12
3/3 [==============================] - 0s 2ms/step - loss: 0.5730 - accuracy: 0.6854
Epoch 10/12
3/3 [==============================] - 0s 2ms/step - loss: 0.5631 - accuracy: 0.6966
Epoch 11/12
3/3 [==============================] - 0s 1ms/step - loss: 0.5537 - accuracy: 0.7416
Epoch 12/12
1/1 [==============================] 

3/3 [==============================] - 0s 1ms/step - loss: 0.5573 - accuracy: 0.7191
Epoch 13/18
3/3 [==============================] - 0s 2ms/step - loss: 0.5440 - accuracy: 0.7303
Epoch 14/18
3/3 [==============================] - 0s 2ms/step - loss: 0.5325 - accuracy: 0.7416
Epoch 15/18
3/3 [==============================] - 0s 3ms/step - loss: 0.5220 - accuracy: 0.7640
Epoch 16/18
3/3 [==============================] - 0s 3ms/step - loss: 0.5106 - accuracy: 0.7753
Epoch 17/18
3/3 [==============================] - 0s 5ms/step - loss: 0.5004 - accuracy: 0.7640
Epoch 18/18
1/1 [==============================] - 0s 905us/step - loss: 0.7191 - accuracy: 0.6364
Epoch 1/18
3/3 [==============================] - 0s 1ms/step - loss: 0.6779 - accuracy: 0.5730
Epoch 2/18
3/3 [==============================] - 0s 2ms/step - loss: 0.6292 - accuracy: 0.6404
Epoch 3/18
3/3 [==============================] - 0s 2ms/step - loss: 0.6040 - accuracy: 0.7079
Epoch 4/18
3/3 [===========================

3/3 [==============================] - 0s 2ms/step - loss: 0.5698 - accuracy: 0.7614
Epoch 17/24
3/3 [==============================] - 0s 2ms/step - loss: 0.5626 - accuracy: 0.7841
Epoch 18/24
3/3 [==============================] - 0s 2ms/step - loss: 0.5553 - accuracy: 0.7841
Epoch 19/24
3/3 [==============================] - 0s 2ms/step - loss: 0.5472 - accuracy: 0.7955
Epoch 20/24
3/3 [==============================] - 0s 2ms/step - loss: 0.5409 - accuracy: 0.7955
Epoch 21/24
3/3 [==============================] - 0s 2ms/step - loss: 0.5320 - accuracy: 0.7955
Epoch 22/24
3/3 [==============================] - 0s 2ms/step - loss: 0.5246 - accuracy: 0.8068
Epoch 23/24
3/3 [==============================] - 0s 2ms/step - loss: 0.5172 - accuracy: 0.8182
Epoch 24/24
1/1 [==============================] - 0s 1ms/step - loss: 0.5234 - accuracy: 0.7826
Epoch 1/24
3/3 [==============================] - 0s 1ms/step - loss: 0.7056 - accuracy: 0.4944
Epoch 2/24
3/3 [===========================

3/3 [==============================] - 0s 2ms/step - loss: 0.6655 - accuracy: 0.6067
Epoch 3/24
3/3 [==============================] - 0s 2ms/step - loss: 0.6437 - accuracy: 0.6629
Epoch 4/24
3/3 [==============================] - 0s 1ms/step - loss: 0.6244 - accuracy: 0.6854
Epoch 5/24
3/3 [==============================] - 0s 3ms/step - loss: 0.6088 - accuracy: 0.6854
Epoch 6/24
3/3 [==============================] - 0s 2ms/step - loss: 0.5956 - accuracy: 0.6742
Epoch 7/24
3/3 [==============================] - 0s 2ms/step - loss: 0.5830 - accuracy: 0.7079
Epoch 8/24
3/3 [==============================] - 0s 4ms/step - loss: 0.5697 - accuracy: 0.7528
Epoch 9/24
3/3 [==============================] - 0s 3ms/step - loss: 0.5589 - accuracy: 0.7640
Epoch 10/24
3/3 [==============================] - 0s 3ms/step - loss: 0.5489 - accuracy: 0.7978
Epoch 11/24
3/3 [==============================] - 0s 2ms/step - loss: 0.5381 - accuracy: 0.8202
Epoch 12/24
3/3 [==============================] 

4/4 [==============================] - 0s 1ms/step - loss: 0.4404 - accuracy: 0.8649
Epoch 21/24
4/4 [==============================] - 0s 2ms/step - loss: 0.4303 - accuracy: 0.8829
Epoch 22/24
4/4 [==============================] - 0s 4ms/step - loss: 0.4210 - accuracy: 0.8829
Epoch 23/24
4/4 [==============================] - 0s 2ms/step - loss: 0.4112 - accuracy: 0.8829
Epoch 24/24
4/4 [==============================] - 0s 2ms/step - loss: 0.4019 - accuracy: 0.9009
LR:  0.8362326210000077
RF:  4.186731511000005
DL:  37.307557972


In [9]:
def train_on_entire(X,y,pipe,grid_res):
    # fit pipeline
    pipe.set_params(**grid_res.best_params_).fit(X, y)
    # return the newly trained pipeline
    return pipe

In [10]:
start = time.perf_counter()
print("Accuracy: ",train_on_entire(X_train,y_train,my_pipe1,my_grid1).score(X_test,y_test))
finish = time.perf_counter()
print("Execution Time for LR: ",finish-start)

Accuracy:  0.6071428571428571
Execution Time for LR:  0.021890389999981608


In [11]:
start = time.perf_counter()
print("Accuracy: ",train_on_entire(X_train,y_train,my_pipe2,my_grid2).score(X_test,y_test))
finish = time.perf_counter()
print("Execution Time for RF: ",finish-start)

Accuracy:  0.6785714285714286
Execution Time for RF:  0.1586094360000061


In [12]:
start = time.perf_counter()
print("Accuracy: ",train_on_entire(X_train,y_train,my_pipe3,my_grid3).score(X_test,y_test))
finish = time.perf_counter()
print("Execution Time for DL: ",finish-start)

Epoch 1/24
4/4 [==============================] - 0s 1ms/step - loss: 0.7451 - accuracy: 0.4414
Epoch 2/24
4/4 [==============================] - 0s 2ms/step - loss: 0.6974 - accuracy: 0.5135
Epoch 3/24
4/4 [==============================] - 0s 2ms/step - loss: 0.6683 - accuracy: 0.5405
Epoch 4/24
4/4 [==============================] - 0s 3ms/step - loss: 0.6507 - accuracy: 0.5856
Epoch 5/24
4/4 [==============================] - 0s 4ms/step - loss: 0.6306 - accuracy: 0.6036
Epoch 6/24
4/4 [==============================] - 0s 2ms/step - loss: 0.6147 - accuracy: 0.6577
Epoch 7/24
4/4 [==============================] - 0s 4ms/step - loss: 0.6001 - accuracy: 0.6486
Epoch 8/24
4/4 [==============================] - 0s 2ms/step - loss: 0.5872 - accuracy: 0.6847
Epoch 9/24
4/4 [==============================] - 0s 5ms/step - loss: 0.5744 - accuracy: 0.7117
Epoch 10/24
4/4 [==============================] - 0s 3ms/step - loss: 0.5609 - accuracy: 0.7477
Epoch 11/24
4/4 [======================